### План работы

1. Выбрать два 7-10 минутных видео на английском языке (TED talk/интервью/...) с мужским и женским голосом
2. Выполнить предобработку данных: выделить аудио, сегментировать, привести к нужному  для ASR модели формату, при необходимости применить деноизинг
3. Запустить любые open-source ASR и MT модели. Объяснить выбор моделей.
4. Разобраться в архитектуре модели XTTS https://arxiv.org/pdf/2406.04904 
5. Zero-shot voice cloning: 
   - скопировать репозиторий https://github.com/coqui-ai/TTS
   - сгенерировать аудио для соответствующих текстов на русском языке аудио моделью XTTSv2. В качестве промпта подавать исходные записи на английском для копирования голоса 
   - наложить аудио на видео
6. Few-shot voice cloning:  
   - выбрать голос персонажа/актера, на котором zero-shot генерация дает не совсем точное копирование/не передает специфические характеристики, и собрать аудиозаписи для него
   - выполнить файнтюнинг модели на этих данных (проверить на разном количестве данных, подобрать гиперпараметры при необходимости)
   - сгенерировать те же предложения до и после файнтюнинга
   - посчитать объективную метрику похожести голоса (speaker smilarity): косинусное расстояние между векторами модели исходной (gt) и сгенерированной записи для zero- и few-shot вариантов

Результат представить:
  - в виде ноутбука с поэтапным описанием каждого шага/ скриптов
  - описания XTTS модели, основных компонент архитектуры и их задач. описание того, какие части обновляются во время few-shot клонирования
  - описания экспериментов по файнтюнингу, текущих проблем пайплайна
  - исходные и полученные видео

### Предобработка

In [1]:
import os
import librosa
import torch
import datetime
import json
from moviepy import VideoFileClip
from pathlib import Path
from tqdm.notebook import tqdm
from transformers import pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [7]:
def convert_video_to_audio_moviepy(video_file: str, output_ext: str = "mp3") -> None:
    """Converts video to audio using MoviePy library
    that uses `ffmpeg` under the hood"""
    filename, _ = os.path.splitext(video_file)
    clip = VideoFileClip(video_file)
    clip.audio.write_audiofile(f"{filename}.{output_ext}")

In [ ]:
for video_file in tqdm(list(Path("data").glob("*.mp4"))):
    convert_video_to_audio_moviepy(video_file)

  0%|          | 0/6 [00:00<?, ?it/s]

MoviePy - Writing audio in data/ch_broken.mp3


MoviePy - Done.
MoviePy - Writing audio in data/Tom Cruise on Performing His Own Dangerous Stunts, ‘Mission Impossible’ Training, and Cliff-Hangers (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/Cameron's interview - CANADA - #HUMAN (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/1987 on TODAY- Bruce Willis talks balancing fame and privacy (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/Bruno's interview - ENGLAND - #HUMAN (360p).mp3


MoviePy - Done.
MoviePy - Writing audio in data/James May Talks- TV, Animals, Clarkson & Hammond (360p).mp3


MoviePy - Done.


### ASR

In [20]:
asr_model = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device
    # "automatic-speech-recognition", model="openai/whisper-small.en", device=device
)

Device set to use cpu


In [ ]:
def speech_to_text(model, audio_file):
    array, sampling_rate = librosa.load(audio_file)
    # trimmed_array, _ = librosa.effects.trim(array)
    intervals = librosa.effects.split(array)
    result = []
    for interval in intervals:
        result.append(
            {
                **model(
                    {
                        "array": array[interval[0]:interval[1]],
                        "sampling_rate": sampling_rate
                    },
                    chunk_length_s=30,
                ),
                "timestamp": librosa.samples_to_time(interval).tolist()
            }
        )
    return {"chunks": result}


In [ ]:
asr_model()

In [144]:
list(Path("data").glob("*.mp3"))[1].stem

'Tom Cruise on Performing His Own Dangerous Stunts, ‘Mission Impossible’ Training, and Cliff-Hangers (360p)'

In [ ]:
result = {}
for audio_file in tqdm(list(Path("data").glob("*.mp3"))):
    result[audio_file.stem] = speech_to_text(asr_model, audio_file)

  0%|          | 0/5 [00:00<?, ?it/s]

/Users/d.kulemin/Projects/dls-video-dubbing/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [189]:
with open("stt_data.json", "w") as fout:
    json.dump(result, fout, sort_keys=True, indent=4)

### MT

In [ ]:
mt_model = pipeline(
    # model="Helsinki-NLP/opus-mt-en-ru", # bad
    # model="facebook/nllb-200-distilled-600M", # not working
    # model="facebook/wmt19-en-ru", # norm
    # model="jbochi/madlad400-7b-mt", # too large
    # model="facebook/wmt21-dense-24-wide-en-x",
    model="google/madlad400-3b-mt",  # norm!
    device=device
)

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/101k [00:00<?, ?B/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/3.34G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cpu


In [4]:
stt_data = json.load(open("stt_data.json"))

In [ ]:
for title, data in tqdm(stt_data.items()):
    for chunk in tqdm(data["chunks"]):
        chunk["translated"] = " ".join([
            output["translation_text"]
            for output in mt_model([sentence + "." for sentence in chunk["text"].split(".")])
        ])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Your input_length: 669 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 203 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 655 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


  0%|          | 0/1 [00:00<?, ?it/s]

Your input_length: 265 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [27]:
with open("mt_data.json", "w", encoding="utf-8") as fout:
    json.dump(stt_data, fout, sort_keys=True, indent=4, ensure_ascii=False)